# Test simple DE with Sharp-ND-R250A5 data

Data are collected in [1]. Ref. [2] has also played with the data using GAMS.

Reference:
- [1] Nunes, H. G. G., et al. "A new high performance method for determining the parameters of PV cells and modules based on guaranteed convergence particle swarm optimization." Applied energy 211 (2018): 774-791.
- [2] Gnetchejo, Patrick Juvet, et al. "Important notes on parameter estimation of solar photovoltaic cell." Energy Conversion and Management 197 (2019): 111870.

In [56]:
using Random, Printf
using CSV, Tables
using Statistics
using DifferentialEvolution

Random.seed!(666)

MersenneTwister(666)

In [57]:
const data = CSV.File("data/Sharp-ND-R250A5.csv") |> Tables.matrix;
@show size(data);

size(data) = (36, 2)


In [58]:
const T = 59 + 273.15;  # temperature in Kelvin 

In [59]:
include("models.jl")  # import related functions

calculate_mae

In [60]:
const np = 50  # number of individuals
const F = 0.6
const Cr = 0.9
const G_sdm = 800 # number of generations
const G_ddm = 2000 # number of generations
;

Note that the PV module has 60 cells in series [2]. The report of parameter $n$ reported in [1] and [2] refer to a single cell.

In [61]:
# search range: same as Table 17 of [2]
bounds_sdm = [
    0 10;
    1e-6 10;
    0.5 * 60 2.5 * 60;
    1e-3 2;
    1e-3 5000
];

In [62]:
# given a parameter vector θ, compute its SSE as the fitness in DE
evaluator(θ) = calculate_sse(data, (V, I) -> model_sdm(V, I, θ..., T));

In [63]:
θ = [9.146451, 1.087673, 1.213178 * 60, 0.589518, 5000]  # from [2]
sse = evaluator(θ)
rmse = sqrt(sse / size(data, 1))
@printf("Min RMSE of Ref. [2] = %.5e", rmse)

Min RMSE of Ref. [2] = 1.12520e-02

In [64]:
# each run is new due to the involved randomness
de = StdDE(np, bounds_sdm, StdOptions(F, Cr); senses=-1)  # minimization
evolve!(de, evaluator, G_sdm; stats=[:best, :std])  # show the best fitness and the standard deviation among population's fitness

gen  	best      	std       	
0    	3.46510e+01	1.21916e+23	
1    	3.46510e+01	2.40518e+12	
2    	3.46510e+01	8.23069e+02	
3    	3.46510e+01	1.52825e+02	
4    	3.46510e+01	1.36875e+02	
5    	3.46510e+01	6.47044e+01	
6    	3.46510e+01	6.60462e+01	
7    	3.46510e+01	6.14141e+01	
8    	3.46510e+01	7.19494e+01	
9    	3.46510e+01	7.32230e+01	
10   	3.46510e+01	7.20362e+01	
11   	3.46510e+01	7.13055e+01	
12   	3.26534e+01	7.28987e+01	
13   	8.66252e+00	7.28467e+01	
14   	2.18303e+00	7.22621e+01	
15   	2.18303e+00	5.39428e+01	
16   	2.18303e+00	4.03823e+01	
17   	2.18303e+00	2.99258e+01	
18   	1.91244e+00	2.77453e+01	
19   	1.91244e+00	2.24773e+01	
20   	1.91244e+00	1.78163e+01	
21   	1.91244e+00	1.68203e+01	
22   	1.91244e+00	1.48069e+01	
23   	1.91244e+00	1.05765e+01	
24   	1.87160e+00	7.57701e+00	
25   	1.00515e+00	5.68195e+00	
26   	3.61272e-01	4.37650e+00	
27   	3.50507e-01	4.17931e+00	
28   	3.50507e-01	3.39066e+00	
29   	3.50507e-01	2.53605e+00	
30   	3.50507e-01	2.17494e+00	
31   	3.50

0×0 Matrix{Float64}

In [65]:
# note that the fitness refers to the SSE
best_sse = best_fitness(de)
best_rmse = sqrt(best_sse / size(data, 1))
@printf("Min RMSE = %.5e", best_rmse)

Min RMSE = 1.12520e-02

In [67]:
println("Best parameter values (SDM + Sharp ND):")
best = best_individual(de)
best[3] /= 60 # turn to the `n` of a single cell for comparison
best

Best parameter values (SDM + Sharp ND):


5-element Vector{Float64}:
    9.146450573121268
    1.087673042035392
    1.2131777247863313
    0.5895182555719707
 4999.999999992114